In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers
import tensorflow as tf
import os
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Comment the following lines if imadedb and imagedb_test are already in content folder or specify path for data in drive
local_zip = '/content/drive/MyDrive/Colab_Notebooks/CV_2020-2021_DL/HW4/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
train_dir = "/content/imagedb" 
test_dir = "/content/imagedb_test" 

In [ ]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', 
                 include_top=False, 
                 input_shape=(128, 128, 3))

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
from keras.backend import tanh,softplus
def mish(x):
    return x * tanh(softplus(x))
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'mish':mish})

In [ ]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f5e95228cf8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5e9c160e10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5e9c160358> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f5e9c1d7f28> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5e95265320> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5e952320b8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f5ece551a58> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5ece552a58> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5ece553780> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5ece5539b0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f5e951234a8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [ ]:
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dense(512,activation="mish"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256,activation="mish"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128,activation="mish"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(34,activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,vertical_flip=True,zoom_range=0.3,rotation_range=20, validation_split=0.2)
# train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_datagen  = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# --------------------
# Flow training images in batches of  using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    # color_mode='grayscale',
                                                    target_size=(128,128),
                                                    shuffle=True,
                                                    subset='training', seed=1)     
# --------------------
# Flow validation images in batches of  using test_datagen generator
# --------------------
validation_generator =  train_datagen.flow_from_directory(train_dir,
                                                        batch_size=128,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(128,128),
                                                        subset='validation', seed=1) 

Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


In [ ]:
import datetime


callbacks = []

logdir = os.path.join("/content/drive/MyDrive/Colab_Notebooks/CV_2020-2021_DL/HW4/logs-pre", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
callbacks.append(tensorboard_callback)

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/Colab_Notebooks/CV_2020-2021_DL/HW4/best_weights_mish_aug_pretrained.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)



In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=150,
      validation_data=validation_generator,
      verbose=1,
      callbacks=callbacks)
 
# Save the model
# model.save('/content/drive/MyDrive/Colab_Notebooks/CV_2020-2021_DL/HW4/mish_aug.h5')

Epoch 1/150
19/19 [==============================] - 24s 847ms/step - loss: 3.3082 - acc: 0.2030 - val_loss: 2.2916 - val_acc: 0.5726

Epoch 00001: val_loss improved from inf to 2.29157, saving model to /content/drive/MyDrive/Colab_Notebooks/CV_2020-2021_DL/HW4/best_weights_mish_aug_pretrained.hdf5
Epoch 2/150
19/19 [==============================] - 12s 636ms/step - loss: 1.3376 - acc: 0.6613 - val_loss: 1.6503 - val_acc: 0.7813

Epoch 00002: val_loss improved from 2.29157 to 1.65029, saving model to /content/drive/MyDrive/Colab_Notebooks/CV_2020-2021_DL/HW4/best_weights_mish_aug_pretrained.hdf5
Epoch 3/150
19/19 [==============================] - 12s 644ms/step - loss: 0.8447 - acc: 0.8071 - val_loss: 1.2691 - val_acc: 0.8748

Epoch 00003: val_loss improved from 1.65029 to 1.26912, saving model to /content/drive/MyDrive/Colab_Notebooks/CV_2020-2021_DL/HW4/best_weights_mish_aug_pretrained.hdf5
Epoch 4/150
19/19 [==============================] - 12s 639ms/step - loss: 0.5399 - acc: 0.

In [ ]:
test_datagen  = ImageDataGenerator(rescale=1./255)
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                        batch_size=64,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(128,128)) 
loss, acc = model.evaluate(test_generator)

Found 2149 images belonging to 34 classes.
34/34 [==============================] - 5s 113ms/step - loss: 0.1191 - acc: 0.9716
